In [1]:
import torch
from torch import nn

import torchvision
from torchvision.datasets import ImageFolder

from torchvision import transforms

from torch.utils.data import DataLoader
from pathlib import Path
from torchvision.models import resnet101

In [2]:
import sys
sys.path.append("..")

In [3]:
from video_classification.datasets import FolderOfFrameFoldersDataset, FrameWindowDataset

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
ROOT = Path("/home/ubuntu/SupervisedVideoClassification")
DATA_ROOT = Path(ROOT/"data")

In [6]:
train_transforms = transforms.Compose([
    torchvision.transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(p=0.25),
    transforms.RandomVerticalFlip(p=0.25),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

In [7]:
train_ds = FolderOfFrameFoldersDataset(DATA_ROOT/'train', 
                                       transform=train_transforms, 
                                       base_class=FrameWindowDataset,
                                       window_size=3,
                                       overlapping=True,)
valid_ds = FolderOfFrameFoldersDataset(DATA_ROOT/'validation', 
                                       transform=valid_transforms, 
                                       base_class=FrameWindowDataset,
                                       window_size=3,
                                       overlapping=True,)

In [8]:
from torch import nn
from torchvision.models import resnet101
from video_classification.models.mlp import MLP


class SingleImageResNetModel(nn.Module):
    def __init__(self, mlp_sizes=[768, 128, 2]):
        super().__init__()
        resnet = resnet101(pretrained=True)
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)

        self.clf = MLP(2048, mlp_sizes)
        self.freeze_resnet()

    def forward(self, x):
        x = self.resnet(x).squeeze()
        x = self.clf(x)
        return x

    def freeze_resnet(self):
        for p in self.resnet.parameters():
            p.requires_grad = False

    def unfreeze_resnet(self):
        for p in self.resnet.parameters():
            p.requires_grad = True


In [9]:
import torch
from torch import nn
from video_classification.models.mlp import MLP


class MultiImageModel(nn.Module):
    def __init__(self,
                 window_size=3,
                 single_mlp_sizes=[768, 128],
                 joint_mlp_sizes=[64, 2]):
        super().__init__()
        self.window_size = window_size
        self.single_mlp_sizes = single_mlp_sizes
        self.joint_mlp_sizes = joint_mlp_sizes
        
        self.single_image_model = SingleImageResNetModel(self.single_mlp_sizes)
        self.in_features = self.single_mlp_sizes[-1] * self.window_size
        self.clf = MLP(self.in_features, joint_mlp_sizes)

    def forward(self, x):
        # x is of size [B, T, C, H, W]. In other words, a batch of windows.
        # each img for the same window goes through SingleImageModel
        x = x.transpose(0, 1)  # -> [T, B, C, H, W]
        x = torch.cat([self.single_image_model(window) for window in x], 1)
        # x is now of size [B, T * single_mlp_sizes[-1]]
        
        x = self.clf(x)
        # Now size is [B, joint_mlp_sizes[-1]] which should always be 2

        return x
    
    def freeze_single_image_model(self):
        # Freeze the VGG classifier
        for p in self.single_image_model.parameters():
            p.requires_grad = False
            
    def unfreeze_single_image_model(self):
        # Unfreeze the VGG classifier. Training the whole VGG is a no-go, so we only train the classifier part.
        for p in self.single_image_model.clf.parameters():
            p.requires_grad = True 

In [10]:
model = MultiImageModel(
                 window_size=3,
                 single_mlp_sizes=[1024, 256],
                 joint_mlp_sizes=[128, 2])

model = model.to(device)

In [11]:
x = torch.stack([train_ds[0][0], train_ds[1][0], train_ds[2][0], train_ds[3][0]]).to(device)

In [12]:
model(x)

tensor([[-0.1784,  0.2266],
        [-0.7380, -0.4095],
        [ 0.2799, -0.2015],
        [ 0.2494, -0.8661]], device='cuda:0', grad_fn=<AddmmBackward>)

In [13]:
from video_classification.trainer import Trainer

classes_weights = torch.Tensor([0.3, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=classes_weights)

In [14]:
trainer = Trainer(train_ds, 
                  valid_ds, 
                  model, 
                  criterion,
                  "multi_frame_resnet101_from_scratch",
                  str(ROOT/'checkpoints'),
                  device=device,
                  amp_opt_level="O1",
                  cycle_mult=0.9,
                 )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [15]:
trainer.train(lr=1e-3, 
              batch_size=128, 
              n_epochs=20,
              gradient_accumulation_steps=2,
              num_workers=8,
              max_gradient_norm=2.0,
             )

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


Training Results - Epoch: 1: Avg accuracy: 0.94 |Precision: 0.95, 0.61 |Recall: 0.98, 0.40 | F1: 0.72 | Avg loss: 0.38
Validation Results - Epoch: 1: Avg accuracy: 0.93 |Precision: 0.93, 0.90 |Recall: 1.00, 0.28 | F1: 0.70 | Avg loss: 0.46


Training Results - Epoch: 2: Avg accuracy: 0.95 |Precision: 0.96, 0.84 |Recall: 0.99, 0.47 | F1: 0.79 | Avg loss: 0.29
Validation Results - Epoch: 2: Avg accuracy: 0.93 |Precision: 0.94, 0.74 |Recall: 0.99, 0.30 | F1: 0.70 | Avg loss: 0.39


Training Results - Epoch: 3: Avg accuracy: 0.93 |Precision: 0.97, 0.55 |Recall: 0.96, 0.63 | F1: 0.78 | Avg loss: 0.27
Validation Results - Epoch: 3: Avg accuracy: 0.88 |Precision: 0.94, 0.34 |Recall: 0.93, 0.35 | F1: 0.64 | Avg loss: 0.47


Training Results - Epoch: 4: Avg accuracy: 0.88 |Precision: 0.98, 0.35 |Recall: 0.89, 0.72 | F1: 0.70 | Avg loss: 0.35
Validation Results - Epoch: 4: Avg accuracy: 0.74 |Precision: 0.95, 0.18 |Recall: 0.76, 0.56 | F1: 0.56 | Avg loss: 0.70


...

Training Results - Epoch: 16: Avg accuracy: 0.97 |Precision: 0.98, 0.83 |Recall: 0.99, 0.77 | F1: 0.89 | Avg loss: 0.14
Validation Results - Epoch: 16: Avg accuracy: 0.94 |Precision: 0.95, 0.70 |Recall: 0.98, 0.50 | F1: 0.77 | Avg loss: 0.38


Training Results - Epoch: 17: Avg accuracy: 0.97 |Precision: 0.98, 0.86 |Recall: 0.99, 0.76 | F1: 0.89 | Avg loss: 0.14
Validation Results - Epoch: 17: Avg accuracy: 0.93 |Precision: 0.96, 0.65 |Recall: 0.97, 0.56 | F1: 0.78 | Avg loss: 0.38


Training Results - Epoch: 18: Avg accuracy: 0.96 |Precision: 0.99, 0.71 |Recall: 0.97, 0.85 | F1: 0.88 | Avg loss: 0.13
Validation Results - Epoch: 18: Avg accuracy: 0.92 |Precision: 0.95, 0.55 |Recall: 0.96, 0.46 | F1: 0.73 | Avg loss: 0.48


Training Results - Epoch: 19: Avg accuracy: 0.97 |Precision: 0.99, 0.78 |Recall: 0.98, 0.84 | F1: 0.90 | Avg loss: 0.12
Validation Results - Epoch: 19: Avg accuracy: 0.91 |Precision: 0.96, 0.50 |Recall: 0.95, 0.57 | F1: 0.74 | Avg loss: 0.40


Training Results - Epoch: 20: Avg accuracy: 0.95 |Precision: 0.99, 0.64 |Recall: 0.96, 0.85 | F1: 0.85 | Avg loss: 0.14
Validation Results - Epoch: 20: Avg accuracy: 0.90 |Precision: 0.96, 0.45 |Recall: 0.93, 0.62 | F1: 0.73 | Avg loss: 0.42


In [16]:
import pandas as pd
reform = {(outerKey, innerKey): values for outerKey, innerDict in trainer.epoch_state.items() for innerKey, values in innerDict.items()}
df = pd.DataFrame(reform).T
df

accuracy        f1       nll  \
1  train  0.936958  0.723321  0.381039   
   test    0.93398  0.698448  0.455625   
2  train  0.954407  0.788925  0.288283   
   test   0.929129  0.696361  0.387364   
3  train  0.934519  0.776292   0.26769   
   test   0.881881  0.640437  0.470022   
4  train  0.880296  0.700706  0.347542   
   test   0.739085  0.557308  0.702977   
5  train  0.962062  0.851589  0.206908   
   test   0.901919  0.672527   0.46011   
6  train  0.959811  0.821811  0.263137   
   test   0.926809  0.708014   0.44321   
7  train  0.924725  0.778395  0.233971   
   test   0.908669  0.732888  0.348829   
8  train  0.963638  0.842913  0.215154   
   test   0.923434  0.733638  0.422452   
9  train   0.95527   0.84023  0.191698   
   test   0.912466  0.719586   0.43496   
10 train  0.943938  0.828908  0.183182   
   test    0.91563  0.733596  0.379819   
11 train  0.950505  0.840155  0.167205   
   test   0.840962  0.639284  0.521018   
12 train    0.9242  0.776527    0.2359   
   test   0.932926  0.794521   0.33281   
13 train   0.96529  0.869421  0.167939   
   test   0.918583  0.723925  0.438031   
14 train  0.966753  0.867621   0.18252   
   test   0.933558  0.741699  0.499725   
15 train  0.968029  0.885385  0.138624   
   test   0.924278   0.75688  0.380356   
16 train  0.971331  0.891704  0.141106   
   test   0.936722  0.773971  0.383102   
17 train  0.972682  0.894124   0.13829   
   test   0.934613  0.783974  0.376983   
18 train  0.963338  0.876921  0.131934   
   test   0.918794  0.728867  0.478138   
19 train  0.970318  0.895193  0.122849   
   test   0.912255  0.742084  0.395623   
20 train  0.954145  0.854052  0.144967   
   test   0.899599  0.732524   0.42257   

                                          precision  \
1  train      [0.953193333595997, 0.61198738170347]   
   test    [0.9349099587763072, 0.9015151515151515]   
2  train   [0.9591158059467919, 0.8438934802571166]   
   test    [0.9361050328227571, 0.7426900584795322]   
3  train   [0.9705447981621267, 0.5481742190937088]   
   test   [0.9370353159851301, 0.33867276887871856]   
4  train   [0.9754058477462733, 0.3483424047501237]   
   test   [0.9461961238067689, 0.18146417445482865]   
5  train    [0.974314794037506, 0.7814530419373893]   
   test    [0.9395942557556417, 0.4350282485875706]   
6  train    [0.9645498898331759, 0.862124898621249]   
   test    [0.9394207384479328, 0.6651376146788991]   
7  train  [0.9801007663321902, 0.49306930693069306]   
   test    [0.9559202813599063, 0.4852941176470588]   
8  train   [0.9682144831398563, 0.8760393046107332]   
   test                [0.9477493779687853, 0.5875]   
9  train   [0.9779468608861944, 0.6862934362934363]   
   test    [0.9489772466099747, 0.5051282051282051]   
10 train   [0.9868990132269578, 0.5829216654904729]   
   test    [0.9520516366989396, 0.5235732009925558]   
11 train   [0.9846511242014436, 0.6268187180495478]   
   test    [0.950050454086781, 0.28442728442728443]   
12 train  [0.9796419350742794, 0.49073461283917935]   
   test     [0.9645011600928074, 0.617169373549884]   
13 train   [0.9790708928139366, 0.7806591031874662]   
   test    [0.9472727272727273, 0.5483870967741935]   
14 train   [0.9755044955044955, 0.8318965517241379]   
   test    [0.9447280799112098, 0.7203389830508474]   
15 train    [0.984548448745578, 0.7704280155642024]   
   test    [0.9552033080634046, 0.5773195876288659]   
16 train   [0.9819867015917791, 0.8271537622682661]   
   test    [0.9527239981990094, 0.6989966555183946]   
17 train   [0.9806974597696537, 0.8572254335260115]   
   test     [0.9582000913659205, 0.650137741046832]   
18 train   [0.9879394089075492, 0.7095541401273885]   
   test    [0.9487121039434694, 0.5480225988700564]   
19 train    [0.987115713936231, 0.7762600094206312]   
   test    [0.9573770491803278, 0.5031847133757962]   
20 train   [0.9881413057046561, 0.6418195718654435]   
   test    [0.9615938550168027, 0.4504347826086956]   

                               